In [ ]:
import pandas as pd
import scanpy as sc
import kitchen.ingredients as k
import dropkick as dk
import MILWRM.ST as st
import sys; sys.path.append("../resources/ST/")
from visium_utils import deconvolve_cnmf

# some stuff to make this notebook work better with Scanpy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# make output directory
import os
if not os.path.exists("ST_out"):
    os.mkdir("ST_out")

if not os.path.exists("ST_out/MILWRM_VUMC_refNMF"):
    os.mkdir("ST_out/MILWRM_VUMC_refNMF")

---
### Read in key dataframe with sample information

In [ ]:
sample_key = pd.read_csv("../resources/ST/visium_sample_key.csv", index_col=0)

In [ ]:
sample_key.columns

In [ ]:
# how to rename usage columns to describe cell states
rename_dict = {
    "usage_1":"usage_1_STM",
    "usage_2":"usage_2_END1",
    "usage_3":"usage_3_BL1",
    "usage_4":"usage_4_FIB1",
    "usage_5":"usage_5_CRC1",
    "usage_6":"usage_6_MYE1",
    "usage_7":"usage_7_TL1",
    "usage_8":"usage_8_MYE2",
    "usage_9":"usage_9_CRC2",
    "usage_10":"usage_10_CT",
    "usage_11":"usage_11_SSC",
    "usage_12":"usage_12_CRC3",
    "usage_13":"usage_13_EE1",
    "usage_14":"usage_14_MYE3",
    "usage_15":"usage_15_PLA",
    "usage_16":"usage_16_FIB2",
    "usage_17":"usage_17_MYE4",
    "usage_18":"usage_18_GOB",
    "usage_19":"usage_19_MAS",
    "usage_20":"usage_20_MYE5",
    "usage_21":"usage_21_CRC4",
    "usage_22":"usage_22_ABS",
    "usage_23":"usage_23_TUF",
    "usage_24":"usage_24_FIB3",
    "usage_25":"usage_25_FIB4",
    "usage_26":"usage_26_TL2",
    "usage_27":"usage_27_END2",
    "usage_28":"usage_28_TL3",
    "usage_29":"usage_29_EE2",
    "usage_30":"usage_30_BL2",
}

# features that describe tissue morphology (not immune) for MILWRM
training_features = [
    "usage_1_STM",
    "usage_2_END1",
    "usage_4_FIB1",
    "usage_5_CRC1",
    "usage_9_CRC2",
    "usage_10_CT",
    "usage_11_SSC",
    "usage_12_CRC3",
    "usage_13_EE1",
    "usage_16_FIB2",
    "usage_18_GOB",
    "usage_21_CRC4",
    "usage_22_ABS",
    "usage_23_TUF",
    "usage_24_FIB3",
    "usage_25_FIB4",
    "usage_27_END2",
    "usage_29_EE2",
]

---
### Now infer cNMF usage scores for Visium spots by factorizing with reference consensus gene spectra

In [ ]:
#pitas = []
adatas = []

for s in sample_key.index:
    print("Starting {}:".format(s), end="\n\t")
    a = sc.read(os.path.join("../", sample_key.loc[s, "trimmed_adata"]))  # read in anndata
    # deconvolve with refNMF
    a, spectra, spectra_ref, niter = deconvolve_cnmf(adata=a, cnmf_dir="../resources/scRNA/VUMC_NMF/", k=30)
    a.obs.rename(columns=rename_dict, inplace=True)
    
    # plot refNMF fractions
    #p = assemble_pita(
    #    a,
    #    features=list(rename_dict.values()),
    #    label=list(rename_dict.values()),
    #    save_to="{}_VUMCrefNMF30.png".format(s),
    #    #vmin=a.obs[list(rename_dict.values())].values.min(),
    #    #vmax=a.obs[list(rename_dict.values())].values.max(),
    #    use_rep=".obs",
    #    ncols=5,
    #    #histo="hires_trim",
    #    cmap="viridis",
    #)
    #pitas.append(p)
    
    a.obsm["MILWRM_predictors"] = a.obs[training_features].values
    adatas.append(a)

---
## Run MILWRM Tissue Labeler

In [ ]:
import MILWRM.MILWRM as mw

In [ ]:
tl = mw.st_labeler(adatas)

In [ ]:
tl.prep_cluster_data(
    use_rep="MILWRM_predictors",
    histo=False,
    n_rings=2,
)

In [ ]:
tl.label_tissue_regions(alpha=0.023)

In [ ]:
tl.confidence_score()

In [ ]:
for s, adata in zip(sample_key.index, tl.adatas):
    p = st.assemble_pita(
        adata,
        features=["tissue_ID", "confidence_score"],
        use_rep=".obs",
        save_to="ST_out/MILWRM_VUMC_refNMF/{}_refNMF_MILWRM_r2_k{}.png".format(s, tl.k),
        histo="hires",
        label=["MILWRM Domain", "Confidence Score"],
        cmap="plasma",
    )

In [ ]:
p = tl.plot_feature_loadings(
    titles=["D0","D1","D2","D3","D4","D5","D6","D7"],
    labels=[x.split("_")[-1] for x in training_features],
    nfeatures=18,
    ncols=4,
    save_to="ST_out/MILWRM_VUMC_refNMF/refNMF_MILWRM_r2_k{}_loadings.png".format(tl.k),
)

In [ ]:
p = tl.plot_percentage_variance_explained(
    R_square=True,
    fig_size=(8,7),
    save_to="ST_out/MILWRM_VUMC_refNMF/refNMF_MILWRM_r2_k{}_variance_explained.png".format(tl.k),
)

In [ ]:
p = tl.plot_mse_st(
    ncols=4,
    figsize=(8,4),
    titles=["D0","D1","D2","D3","D4","D5","D6","D7"],
    #labels=[x.split("_")[-1] for x in training_features],
    save_to="ST_out/MILWRM_VUMC_refNMF/refNMF_MILWRM_r2_k{}_MSE.png".format(tl.k),
)

In [ ]:
sample_key["unique_name"] = " (" + sample_key.sample_key_short + ")"
sample_key.loc[sample_key.block_name==sample_key.patient_name, "unique_name"] = ""

In [ ]:
# make unique sample names for tissue ID proportions plot
sample_key.unique_name = sample_key.patient_name + sample_key.unique_name
# synchronous polyps from MAP8622 - fix names
sample_key.loc["7319_2_HTA11_08622_A", "unique_name"] = "HTA11_08622_A"
sample_key.loc["7319_3_HTA11_08622_B", "unique_name"] = "HTA11_08622_B"

In [ ]:
p = tl.plot_tissue_ID_proportions_st(
    tID_labels=["D0","D1","D2","D3","D4","D5","D6","D7"],
    slide_labels=list(sample_key.unique_name),
    figsize=(12,5),
    cmap="plasma",
    save_to="ST_out/MILWRM_VUMC_refNMF/refNMF_MILWRM_r2_k{}_tIDproportions.png".format(tl.k),
)

---
## Rename `.obs` entry and `MILWRM` tissue domains and save to `.h5ad`

In [ ]:
import os

In [ ]:
# dict to rename tissue domains
mapper = {
    "0" : "D0",
    "1" : "D1",
    "2" : "D2",
    "3" : "D3",
    "4" : "D4",
    "5" : "D5",
    "6" : "D6",
    "7" : "D7",
}

for i, s in enumerate(sample_key.index):
    name = os.path.join("../", sample_key.loc[s, "MILWRM_VUMCrefNMF30_adata"])
    print("Saving {} to {}".format(s, name))
    #if "MILWRM Domain" in tl.adatas[i].obs.columns:
    #    tl.adatas[i].obs.drop(columns=["MILWRM Domain", "MILWRM Confidence Score"], inplace=True)
    if "tissue_ID" in tl.adatas[i].obs.columns:
        print("adjusting tissue_ID to refNMF_MILWRM_domain")
        tl.adatas[i].obs.tissue_ID = tl.adatas[i].obs.tissue_ID.astype(str)
        tl.adatas[i].obs.replace({"tissue_ID" : mapper}, inplace=True)
        tl.adatas[i].obs.rename(
            columns={"tissue_ID":"MILWRM Domain", "confidence_score":"MILWRM Confidence Score"},
            inplace=True,
        )
    tl.adatas[i].write(name)